In [6]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Transformations-Actions") \
    .master("spark://dc612074df78:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

25/02/21 14:14:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
import importlib
import team_name.spark_utils

importlib.reload(team_name.spark_utils)

columns_info = [ ("track_id", "string"),
                ("track_name", "string"),
                ("artist_id", "string"),
                ("artist_name", "string"),
                ("album_id", "string"),
                ("duration", "integer"),
                ("release_date", "timestamp"),
                ("popularity", "integer"),
                ("danceability", "double"),
                ("energy", "double"),
                ("key", "integer"),
                ("loudness", "double"),
                ("mode", "integer"),
                ("speechiness", "double"),
                ("acousticness", "double"),
                ("instrumentalness", "double"),
                ("liveness", "double"),
                ("valence", "double"),
                ("tempo", "double"),
                ("playlist_id", "string"),
                ("playlist_name", "string"),
                ("duration_mins", "double"),
                ("genre", "string")]

schema = team_name.spark_utils.SparkUtils.generate_schema(columns_info)

# Create DataFrame
tiktok_df = spark \
                .read \
                .schema(schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/tiktok.csv")

In [9]:
# Filter and Count Popular Tracks. Filter songs with a popularity score greater than 80 and count the number of such tracks.
filtered_duration_df = tiktok_df.filter(tiktok_df["popularity"] > 80)
row_count = filtered_duration_df.count()


25/02/21 14:19:19 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/02/21 14:19:34 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.

KeyboardInterrupt: 

25/02/21 14:19:49 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [ ]:
 # Select only necessary columns
df = df.select("popularity", "duration_mins", "genre", "energy", "playlist_name")
 
# Filter and Count Popular Tracks
popular_tracks = df.filter(col("popularity") > 80)
 
# Calculate Average Duration of Songs by Genre
avg_duration_by_genre = df.groupBy("genre").agg(avg(col("duration_mins")).alias("avg_duration"))
 
# Find the Top 5 Most Energetic Songs
top_energetic_songs = df.orderBy(col("energy").desc()).limit(5)
 
# Calculate the Total Duration of Songs in Each Playlist
total_duration_by_playlist = df.groupBy("playlist_id").agg(sum(col("duration_mins")).alias("total_duration"))
 
# Actions
popular_tracks_count = popular_tracks.count()
print(f"Number of popular tracks: {popular_tracks_count}")
avg_duration_by_genre.show()
top_energetic_songs.show()
total_duration_by_playlist.show()

In [5]:
# Stop the SparkContext
sc.stop()